In [1]:
#!pip install faiss-gpu

In [ ]:
# !pip install gdown

In [4]:
# !pip install faiss-gpu==1.5.3

In [ ]:
# !apt install unrar

In [ ]:
# !gdown --id 1JPBzZHJCT5uO7QgLR7O5dIJNIA0XCGuQ

In [ ]:
# !unrar x ImageNet\(Task3\).rar

In [ ]:
# !git clone https://github.com/facebookresearch/deepcluster.git

In [1]:
# %cd deepcluster/

[Errno 2] No such file or directory: 'deepcluster/'
/home/nova/Lab_individual_works/Daniyal/thesis/Code/deepcluster


In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

In [2]:
!echo $CUDA_VISIBLE_DEVICES

0,1


In [3]:
import argparse
import os
import pickle
import time

import faiss
import numpy as np
from sklearn.metrics.cluster import normalized_mutual_info_score
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets

import clustering
import models
from util import AverageMeter, Logger, UnifLabelSampler

In [4]:
def main(args):
    # fix random seeds
    torch.manual_seed(args.seed)
    torch.cuda.manual_seed_all(args.seed)
    np.random.seed(args.seed)

    # CNN
    if args.verbose:
        print('Architecture: {}'.format(args.arch))
    model = models.__dict__[args.arch](sobel=args.sobel)
    fd = int(model.top_layer.weight.size()[1])
    model.top_layer = None
    model.features = torch.nn.DataParallel(model.features)
    model.cuda()
    cudnn.benchmark = True

    # create optimizer
    optimizer = torch.optim.SGD(
        filter(lambda x: x.requires_grad, model.parameters()),
        lr=args.lr,
        momentum=args.momentum,
        weight_decay=10**args.wd,
    )

    # define loss function
    criterion = nn.CrossEntropyLoss().cuda()

    # optionally resume from a checkpoint
    if args.resume:
        if os.path.isfile(args.resume):
            print("=> loading checkpoint '{}'".format(args.resume))
            checkpoint = torch.load(args.resume)
            args.start_epoch = checkpoint['epoch']
            # remove top_layer parameters from checkpoint
#             for key in checkpoint['state_dict']:
#                 if 'top_layer' in key:
#                     del checkpoint['state_dict'][key]

            del checkpoint['state_dict']['top_layer.weight']
            del checkpoint['state_dict']['top_layer.bias']
            model.load_state_dict(checkpoint['state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer'])
            print("=> loaded checkpoint '{}' (epoch {})"
                  .format(args.resume, checkpoint['epoch']))
        else:
            print("=> no checkpoint found at '{}'".format(args.resume))

    # creating checkpoint repo
    exp_check = os.path.join(args.exp, 'checkpoints')
    if not os.path.isdir(exp_check):
        os.makedirs(exp_check)

    # creating cluster assignments log
    cluster_log = Logger(os.path.join(args.exp, 'clusters'))

    # preprocessing of data
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    tra = [transforms.Resize(256),
           transforms.CenterCrop(224),
           transforms.ToTensor(),
           normalize]

    # load the data
    end = time.time()
    dataset = datasets.ImageFolder(args.data, transform=transforms.Compose(tra))
    if args.verbose:
        print('Load dataset: {0:.2f} s'.format(time.time() - end))

    dataloader = torch.utils.data.DataLoader(dataset,
                                             batch_size=args.batch,
                                             num_workers=args.workers,
                                             pin_memory=True)

    # clustering algorithm to use
    deepcluster = clustering.__dict__[args.clustering](args.nmb_cluster)

    # training convnet with DeepCluster
    for epoch in range(args.start_epoch, args.epochs):
        end = time.time()

        # remove head
        model.top_layer = None
        model.classifier = nn.Sequential(*list(model.classifier.children())[:-1])

        # get the features for the whole dataset
        features = compute_features(dataloader, model, len(dataset))

        # cluster the features
        if args.verbose:
            print('Cluster the features')
        clustering_loss = deepcluster.cluster(features, verbose=args.verbose)

        # assign pseudo-labels
        if args.verbose:
            print('Assign pseudo labels')
        train_dataset = clustering.cluster_assign(deepcluster.images_lists,
                                                  dataset.imgs)

        # uniformly sample per target
        sampler = UnifLabelSampler(int(args.reassign * len(train_dataset)),
                                   deepcluster.images_lists)

        train_dataloader = torch.utils.data.DataLoader(
            train_dataset,
            batch_size=args.batch,
            num_workers=args.workers,
            sampler=sampler,
            pin_memory=True,
        )

        # set last fully connected layer
        mlp = list(model.classifier.children())
        mlp.append(nn.ReLU(inplace=True).cuda())
        model.classifier = nn.Sequential(*mlp)
        model.top_layer = nn.Linear(fd, len(deepcluster.images_lists))
        model.top_layer.weight.data.normal_(0, 0.01)
        model.top_layer.bias.data.zero_()
        model.top_layer.cuda()

        # train network with clusters as pseudo-labels
        end = time.time()
        loss = train(train_dataloader, model, criterion, optimizer, epoch)

        # print log
        if args.verbose:
            print('###### Epoch [{0}] ###### \n'
                  'Time: {1:.3f} s\n'
                  'Clustering loss: {2:.3f} \n'
                  'ConvNet loss: {3:.3f}'
                  .format(epoch, time.time() - end, clustering_loss, loss))
            try:
                nmi = normalized_mutual_info_score(
                    clustering.arrange_clustering(deepcluster.images_lists),
                    clustering.arrange_clustering(cluster_log.data[-1])
                )
                print('NMI against previous assignment: {0:.3f}'.format(nmi))
            except IndexError:
                pass
            print('####################### \n')
        # save running checkpoint
        torch.save({'epoch': epoch + 1,
                    'arch': args.arch,
                    'state_dict': model.state_dict(),
                    'optimizer' : optimizer.state_dict()},
                   os.path.join(args.exp, 'checkpoint.pth.tar'))

        # save cluster assignments
        cluster_log.log(deepcluster.images_lists)

In [5]:
def train(loader, model, crit, opt, epoch):
    """Training of the CNN.
        Args:
            loader (torch.utils.data.DataLoader): Data loader
            model (nn.Module): CNN
            crit (torch.nn): loss
            opt (torch.optim.SGD): optimizer for every parameters with True
                                   requires_grad in model except top layer
            epoch (int)
    """
    batch_time = AverageMeter()
    losses = AverageMeter()
    data_time = AverageMeter()
    forward_time = AverageMeter()
    backward_time = AverageMeter()

    # switch to train mode
    model.train()

    # create an optimizer for the last fc layer
    optimizer_tl = torch.optim.SGD(
        model.top_layer.parameters(),
        lr=args.lr,
        weight_decay=10**args.wd,
    )

    end = time.time()
    for i, (input_tensor, target) in enumerate(loader):
        data_time.update(time.time() - end)

        # save checkpoint
        n = len(loader) * epoch + i
        if n % args.checkpoints == 0:
            path = os.path.join(
                args.exp,
                'checkpoints',
                'checkpoint_' + str(n / args.checkpoints) + '.pth.tar',
            )
            if args.verbose:
                print('Save checkpoint at: {0}'.format(path))
            torch.save({
                'epoch': epoch + 1,
                'arch': args.arch,
                'state_dict': model.state_dict(),
                'optimizer' : opt.state_dict()
            }, path)

        target = target.cuda(non_blocking=True)
        input_var = torch.autograd.Variable(input_tensor.cuda())
        target_var = torch.autograd.Variable(target)

        output = model(input_var)
        loss = crit(output, target_var)

        # record loss
#         losses.update(loss.data[0], input_tensor.size(0)) original
        losses.update(loss.data, input_tensor.size(0))

        # compute gradient and do SGD step
        opt.zero_grad()
        optimizer_tl.zero_grad()
        loss.backward()
        opt.step()
        optimizer_tl.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if args.verbose and (i % 200) == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time: {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data: {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss: {loss.val:.4f} ({loss.avg:.4f})'
                  .format(epoch, i, len(loader), batch_time=batch_time,
                          data_time=data_time, loss=losses))

    return losses.avg

In [6]:
def compute_features(dataloader, model, N):
    if args.verbose:
        print('Compute features')
    batch_time = AverageMeter()
    end = time.time()
    model.eval()
    # discard the label information in the dataloader
    for i, (input_tensor, _) in enumerate(dataloader):
        input_var = torch.autograd.Variable(input_tensor.cuda(), volatile=True)
        aux = model(input_var).data.cpu().numpy()

        if i == 0:
            features = np.zeros((N, aux.shape[1]), dtype='float32')

        aux = aux.astype('float32')
        if i < len(dataloader) - 1:
            features[i * args.batch: (i + 1) * args.batch] = aux
        else:
            # special treatment for final batch
            features[i * args.batch:] = aux

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if args.verbose and (i % 200) == 0:
            print('{0} / {1}\t'
                  'Time: {batch_time.val:.3f} ({batch_time.avg:.3f})'
                  .format(i, len(dataloader), batch_time=batch_time))
    return features

In [16]:
class args:
    def __init__(self, verbose=True, resume=' ', seed=31, lr=0.05, momentum=0.9, wd=-5, start_epoch=0, 
                 exp='', data='', clustering='Kmeans', epochs=500, reassign=1., batch=256, workers=4, 
                 arch='alexnet', checkpoints=25000, sobel=True, nmb_cluster=10000):
        self.verbose = verbose
        self.arch = arch
        self.clustering = clustering
        self.lr = lr
        self.verbose = verbose
        self.resume = resume
        self.seed = seed
        self.momentum = momentum
        self.wd = wd
        self.exp = exp
        self.start_epoch = start_epoch
        self.data = data
        self.batch = batch
        self.epochs = epochs
        self.reassign = reassign
        self.workers = workers
        self.checkpoints = checkpoints
        self.sobel = sobel
        self.nmb_cluster = nmb_cluster

In [17]:
args = args(data='/home/nova/Lab_individual_works/Daniyal/thesis/Data/ILSVRC_TRAIN', exp='exp1')

In [ ]:
main(args)

Architecture: alexnet
=> no checkpoint found at ' '
Load dataset: 3.45 s
Compute features


/home/nova/.local/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  if __name__ == '__main__':


0 / 5005	Time: 1.920 (1.920)
200 / 5005	Time: 1.111 (0.647)
400 / 5005	Time: 2.124 (0.693)
600 / 5005	Time: 0.311 (0.693)
800 / 5005	Time: 0.325 (0.682)
1000 / 5005	Time: 0.354 (0.658)


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f703683fd40>
Traceback (most recent call last):
  File "/home/nova/.local/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/home/nova/.local/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
    if w.is_alive():
  File "/home/nova/anaconda3/envs/deepcluster/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f703683fd40>
Traceback (most recent call last):
  File "/home/nova/.local/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/home/nova/.local/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1320, in 

1200 / 5005	Time: 0.041 (0.637)
1400 / 5005	Time: 0.042 (0.624)
1600 / 5005	Time: 0.498 (0.628)
1800 / 5005	Time: 0.041 (0.625)
2000 / 5005	Time: 1.140 (0.632)
2200 / 5005	Time: 1.363 (0.632)
2400 / 5005	Time: 0.366 (0.633)
2600 / 5005	Time: 0.041 (0.635)
2800 / 5005	Time: 0.123 (0.637)
3000 / 5005	Time: 1.083 (0.640)
3200 / 5005	Time: 0.665 (0.640)
3400 / 5005	Time: 1.763 (0.643)
3600 / 5005	Time: 0.267 (0.644)
3800 / 5005	Time: 0.352 (0.642)
4000 / 5005	Time: 0.225 (0.641)
4200 / 5005	Time: 0.198 (0.642)
4400 / 5005	Time: 1.002 (0.644)
4600 / 5005	Time: 0.658 (0.646)
4800 / 5005	Time: 1.369 (0.649)
5000 / 5005	Time: 0.124 (0.653)
Cluster the features
k-means loss evolution: [1822899.  1126476.5 1111094.8 1104548.8 1100997.1 1098795.2 1097309.4
 1096273.1 1095491.4 1094906.6 1094437.1 1094067.1 1093763.5 1093508.2
 1093300.6 1093132.6 1092979.8 1092856.1 1092753.  1092658.8]
k-means time: 651 s
Assign pseudo labels
Save checkpoint at: exp1/checkpoints/checkpoint_0.0.pth.tar
Epoch: [0]

...

0 / 5005	Time: 2.879 (2.879)
200 / 5005	Time: 1.658 (0.424)
400 / 5005	Time: 1.816 (0.427)
600 / 5005	Time: 0.784 (0.423)
800 / 5005	Time: 0.513 (0.417)
1000 / 5005	Time: 0.041 (0.405)
1200 / 5005	Time: 0.041 (0.402)
1400 / 5005	Time: 0.562 (0.400)
1600 / 5005	Time: 1.166 (0.402)
1800 / 5005	Time: 0.197 (0.401)
2000 / 5005	Time: 0.043 (0.402)
2200 / 5005	Time: 0.041 (0.402)
2400 / 5005	Time: 0.042 (0.403)
2600 / 5005	Time: 0.041 (0.403)
2800 / 5005	Time: 0.041 (0.403)
3000 / 5005	Time: 0.042 (0.404)
3200 / 5005	Time: 0.221 (0.403)
3400 / 5005	Time: 0.415 (0.403)
3600 / 5005	Time: 0.197 (0.403)
3800 / 5005	Time: 0.042 (0.403)
4000 / 5005	Time: 0.475 (0.403)
4200 / 5005	Time: 0.272 (0.403)
4400 / 5005	Time: 0.998 (0.404)
4600 / 5005	Time: 0.052 (0.403)
4800 / 5005	Time: 1.382 (0.404)
5000 / 5005	Time: 1.409 (0.406)
Cluster the features
k-means loss evolution: [1277792.4   865418.4   845599.06  836602.9   831432.6   828102.75
  825748.    824023.2   822709.44  821668.94  820824.44  820132

/home/nova/.local/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  if __name__ == '__main__':


0 / 5005	Time: 2.954 (2.954)
200 / 5005	Time: 1.452 (0.421)
400 / 5005	Time: 1.830 (0.425)
600 / 5005	Time: 0.981 (0.423)
800 / 5005	Time: 1.111 (0.417)
1000 / 5005	Time: 0.409 (0.405)
1200 / 5005	Time: 0.041 (0.402)
1400 / 5005	Time: 0.948 (0.401)
1600 / 5005	Time: 1.274 (0.402)
1800 / 5005	Time: 0.041 (0.401)
2000 / 5005	Time: 0.041 (0.402)
2200 / 5005	Time: 0.042 (0.402)
2400 / 5005	Time: 0.041 (0.403)
2600 / 5005	Time: 0.041 (0.402)
2800 / 5005	Time: 0.041 (0.403)
3000 / 5005	Time: 0.041 (0.404)
3200 / 5005	Time: 0.041 (0.402)
3400 / 5005	Time: 0.880 (0.403)
3600 / 5005	Time: 0.041 (0.404)
3800 / 5005	Time: 0.044 (0.403)
4000 / 5005	Time: 0.372 (0.403)
4200 / 5005	Time: 0.041 (0.403)
4400 / 5005	Time: 0.041 (0.404)
4600 / 5005	Time: 0.041 (0.404)
4800 / 5005	Time: 1.425 (0.405)
5000 / 5005	Time: 1.233 (0.406)
Cluster the features
k-means loss evolution: [1279736.6   865378.5   845545.75  836568.9   831409.7   828051.3
  825708.9   823982.6   822678.94  821628.2   820751.    820074.

/home/nova/.local/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  if __name__ == '__main__':


0 / 5005	Time: 2.952 (2.952)
200 / 5005	Time: 1.630 (0.422)
400 / 5005	Time: 1.808 (0.426)
600 / 5005	Time: 0.582 (0.422)
800 / 5005	Time: 0.231 (0.416)
1000 / 5005	Time: 0.042 (0.405)
1200 / 5005	Time: 0.041 (0.402)
1400 / 5005	Time: 0.042 (0.400)
1600 / 5005	Time: 1.013 (0.401)
1800 / 5005	Time: 0.042 (0.401)
2000 / 5005	Time: 0.041 (0.402)
2200 / 5005	Time: 0.041 (0.402)
2400 / 5005	Time: 0.041 (0.402)
2600 / 5005	Time: 0.041 (0.402)
2800 / 5005	Time: 0.041 (0.403)


In [39]:
checkpoint = torch.load('exp/checkpoint.pth.tar')

In [40]:
for key in checkpoint['state_dict']:
    if "top_layer" in key:
        print(key)

top_layer.weight
top_layer.bias
